In [1]:
import datetime
import stock_marker_binance
import model_design
import pandas as pd
from sklearn.utils import class_weight
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
EPOCHS =  2000
TIME_FRAME = [2, 3, 21]
TRAIN_FRACTION = 0.75

In [3]:
def dataframe_to_tensor(df, timestep_length: int):
    
    '''
    Converts a pandas dataframe to a tensor
    
    Input: Pandas dataframe
            timestep_length (int) - the length of the timestep
    Output: Tensor
    
    '''
    
    # Convert the dataframe to a numpy array
    df = np.array(df)

    n_seq = len(df) - timestep_length + 1
    return np.array([df[i:(i+timestep_length)] for i in range(n_seq)])

In [4]:
def get_train_test_sets(data, train_frac):
    n_train = int(data.shape[0] * train_frac)
    x_train = data[:n_train, :, :-4]
    y_train = data[:n_train, -1:, -4:].reshape(-1,4)
    x_test = data[n_train:, :, :-4]
    y_test = data[n_train:, -1:, -4:].reshape(-1,4)
    return x_train, y_train, x_test, y_test

In [5]:
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [6]:
df = pd.read_csv('ETH_1h_history as of 09-01-2024.csv', index_col=0)

In [7]:
df.head()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades
0,2020-01-01 00:00:00,129.16,129.19,128.68,128.87,7769.17336,1577840399999,1.000930e+06,2504
1,2020-01-01 01:00:00,128.87,130.65,128.78,130.64,11344.65516,1577843999999,1.474278e+06,4885
2,2020-01-01 02:00:00,130.63,130.98,130.35,130.85,7603.35623,1577847599999,9.940256e+05,3046
3,2020-01-01 03:00:00,130.85,130.89,129.94,130.20,4968.55433,1577851199999,6.473610e+05,2818
4,2020-01-01 04:00:00,130.21,130.74,130.15,130.20,3397.90747,1577854799999,4.430067e+05,2264


In [8]:
df_result = stock_marker_binance.sorting_timestamp(df)

df_result['hour_sin'] = np.sin(df_result['timestamp'].dt.hour / 24 * 2 * np.pi)
df_result['hour_cos'] = np.cos(df_result['timestamp'].dt.hour / 24 * 2 * np.pi)

df_result['day_sin'] = np.sin(df_result['timestamp'].dt.day / 30 * 2 * np.pi)
df_result['day_cos'] = np.cos(df_result['timestamp'].dt.day / 30 * 2 * np.pi)

df_result['mon_sin'] = np.sin(df_result['timestamp'].dt.month / 12 * 2 * np.pi)
df_result['mon_cos'] = np.cos(df_result['timestamp'].dt.month / 12 * 2 * np.pi)

df_result['weekday_sin'] = np.sin(df_result['timestamp'].dt.weekday / 7 * 2 * np.pi)
df_result['weekday_cos'] = np.cos(df_result['timestamp'].dt.weekday / 7 * 2 * np.pi)

df_result['year'] = df_result['timestamp'].dt.year


In [9]:
df_result.head(10)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year
0,2020-01-01 00:00:00,129.16,129.19,128.68,128.87,7769.17336,1577840399999,1.000930e+06,2504,0.000000,1.000000e+00,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
1,2020-01-01 01:00:00,128.87,130.65,128.78,130.64,11344.65516,1577843999999,1.474278e+06,4885,0.258819,9.659258e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
2,2020-01-01 02:00:00,130.63,130.98,130.35,130.85,7603.35623,1577847599999,9.940256e+05,3046,0.500000,8.660254e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
3,2020-01-01 03:00:00,130.85,130.89,129.94,130.20,4968.55433,1577851199999,6.473610e+05,2818,0.707107,7.071068e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
4,2020-01-01 04:00:00,130.21,130.74,130.15,130.20,3397.90747,1577854799999,4.430067e+05,2264,0.866025,5.000000e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
5,2020-01-01 05:00:00,130.20,130.47,130.11,130.30,4243.60640,1577858399999,5.529727e+05,2426,0.965926,2.588190e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
6,2020-01-01 06:00:00,130.31,130.75,130.26,130.44,3668.90166,1577861999999,4.789442e+05,2347,1.000000,6.123234e-17,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
7,2020-01-01 07:00:00,130.47,130.71,130.14,130.24,4147.17413,1577865599999,5.407707e+05,2568,0.965926,-2.588190e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
8,2020-01-01 08:00:00,130.24,130.41,129.87,130.36,7541.44497,1577869199999,9.808940e+05,3039,0.866025,-5.000000e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
9,2020-01-01 09:00:00,130.40,130.62,130.13,130.17,4808.20496,1577872799999,6.268190e+05,2696,0.707107,-7.071068e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020


In [10]:

df_result = stock_marker_binance.std(df_result, ['close', 'quote_asset_volume', 'num_trades'])

print("Complete adding STDs")

df_result = stock_marker_binance.adding_MAs(df_result, ['close', 'quote_asset_volume', 'num_trades'])

print("Complete adding MAs")


df_result = stock_marker_binance.adding_ratio(df_result, ['close', 'quote_asset_volume', 'num_trades'], ['MA'])
df_result = stock_marker_binance.adding_ratio(df_result, ['close'], ['STD'])

print("Complete adding ratios")

df_result = stock_marker_binance.rsi(df_result, ['close', 'quote_asset_volume', 'num_trades'])

print("Complete adding RSI")

df_result = stock_marker_binance.marking_buy_sell_actions(df_result)
print("Complete marking buy sell actions")
df_result = stock_marker_binance.marking_adjacent_actions(df_result)
print("Complete marking adjacent actions")
df_result = stock_marker_binance.marking_hold_wait_actions(df_result)
print("Complete marking hold wait actions")

df_result = df_result.drop(columns=['timestamp','open', 'high', 'low' ,'close_time', 'volume',
                                    'close',  'quote_asset_volume', 'num_trades'
                                    ])
df_result=df_result.dropna()
df_result.reset_index(inplace=True, drop=True)
# df_result = stock_marker_binance.renaming_cols(df_result, STOCK_TICKER + "_", ['timestamp', 'action'])

Complete adding STDs
Complete adding MAs
Complete adding ratios
Complete adding RSI


c:\Users\user\Documents\GitHub\BinanceTradingBot\stock_marker_binance.py:272: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['RSI_'+ str(period) + '_' + col_label] = 100.0 - (100.0 / (1.0 + RS))
c:\Users\user\Documents\GitHub\BinanceTradingBot\stock_marker_binance.py:272: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['RSI_'+ str(period) + '_' + col_label] = 100.0 - (100.0 / (1.0 + RS))
c:\Users\user\Documents\GitHub\BinanceTradingBot\stock_marker_binance.py:272: PerformanceWarning: DataFrame is highly fragmented.  This is u

Complete marking buy sell actions
Complete marking adjacent actions
Complete marking hold wait actions


In [11]:
df_result.groupby('action').count()

,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,STD3_close,...,RSI_192_quote_asset_volume,RSI_384_quote_asset_volume,RSI_3_num_trades,RSI_6_num_trades,RSI_12_num_trades,RSI_24_num_trades,RSI_48_num_trades,RSI_96_num_trades,RSI_192_num_trades,RSI_384_num_trades
action,,,,,,,,,,,,,,,,,,,,,
B,2341,2341,2341,2341,2341,2341,2341,2341,2341,2341,...,2341,2341,2341,2341,2341,2341,2341,2341,2341,2341
H,11121,11121,11121,11121,11121,11121,11121,11121,11121,11121,...,11121,11121,11121,11121,11121,11121,11121,11121,11121,11121
S,2650,2650,2650,2650,2650,2650,2650,2650,2650,2650,...,2650,2650,2650,2650,2650,2650,2650,2650,2650,2650
W,18832,18832,18832,18832,18832,18832,18832,18832,18832,18832,...,18832,18832,18832,18832,18832,18832,18832,18832,18832,18832


In [12]:
# Dropping cols
drop_cols_list =[]
for col in df_result.columns:
    if col[:2]=="MA" or col[:3]=="STD":
        drop_cols_list.append(col)
df_result.drop(columns=drop_cols_list, inplace=True)

In [13]:
df_result

,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,RATIO_close_and_MA3,...,RSI_384_quote_asset_volume,RSI_3_num_trades,RSI_6_num_trades,RSI_12_num_trades,RSI_24_num_trades,RSI_48_num_trades,RSI_96_num_trades,RSI_192_num_trades,RSI_384_num_trades,action
0,0.000000,1.000000e+00,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,0.998395,...,50.285725,55.233549,62.375595,44.246383,47.372031,41.390263,50.373816,50.212888,50.352010,B
1,0.258819,9.659258e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.003510,...,50.044557,57.042254,53.231167,43.468397,40.757926,44.977155,50.329082,49.961415,50.023663,S
2,0.500000,8.660254e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.005462,...,50.374138,71.936820,60.894401,54.737831,53.922093,51.046656,50.508626,50.688208,50.405497,W
3,0.707107,7.071068e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,0.999496,...,50.454235,63.487630,59.629177,57.361954,54.634917,49.887067,50.916972,50.645574,50.478997,W
4,0.866025,5.000000e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.005046,...,50.273035,62.021245,59.762526,46.674281,52.189555,47.257474,50.685501,50.334074,50.297334,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34939,0.707107,7.071068e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,0.999538,...,49.917523,0.491800,45.842406,44.909560,46.166448,50.426042,50.048528,49.884557,49.932173,W
34940,0.866025,5.000000e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,1.001322,...,49.923296,9.573140,51.228791,46.818007,49.772624,50.584997,49.856044,49.908062,49.903714,W
34941,0.965926,2.588190e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,0.999021,...,49.684454,70.756458,46.134609,41.865474,49.885167,50.572507,50.125561,50.051341,49.662115,W
34942,1.000000,6.123234e-17,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,0.997117,...,49.859221,71.389892,3.699243,27.179893,46.358171,50.568465,49.547127,50.109445,49.851729,W


In [14]:
# le = sklearn_prep.LabelEncoder()
# df_result['action'] = le.fit_transform(df_result['action'])

enc = OneHotEncoder()

OHE_array = enc.fit_transform(df_result[['action']]).toarray()

class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes=np.unique(df_result['action']),
                                                 y=df_result['action'])



class_weights_dict = dict(zip(enc.categories_[0], class_weights))

print(class_weights)
print(class_weights_dict.keys())


[3.73173857 0.78554087 3.29660377 0.46389125]
dict_keys(['B', 'H', 'S', 'W'])


In [15]:
class_weights_dict=dict(zip(range(0,4), class_weights))

In [16]:
class_weights_dict

{0: 3.7317385732592907,
 1: 0.7855408686269221,
 2: 3.2966037735849056,
 3: 0.4638912489379779}

In [17]:
OHE_df = pd.DataFrame(OHE_array, columns=enc.categories_[0])

In [18]:
df_result = pd.merge(df_result, OHE_df, left_index=True, right_index=True)
df_result.drop(columns=['action'], inplace=True)

In [19]:
print(class_weights_dict)

{0: 3.7317385732592907, 1: 0.7855408686269221, 2: 3.2966037735849056, 3: 0.4638912489379779}


In [20]:
df_result.head()

,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,RATIO_close_and_MA3,...,RSI_12_num_trades,RSI_24_num_trades,RSI_48_num_trades,RSI_96_num_trades,RSI_192_num_trades,RSI_384_num_trades,B,H,S,W
0,0.000000,1.000000,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,0.998395,...,44.246383,47.372031,41.390263,50.373816,50.212888,50.352010,1.0,0.0,0.0,0.0
1,0.258819,0.965926,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.003510,...,43.468397,40.757926,44.977155,50.329082,49.961415,50.023663,0.0,0.0,1.0,0.0
2,0.500000,0.866025,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.005462,...,54.737831,53.922093,51.046656,50.508626,50.688208,50.405497,0.0,0.0,0.0,1.0
3,0.707107,0.707107,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,0.999496,...,57.361954,54.634917,49.887067,50.916972,50.645574,50.478997,0.0,0.0,0.0,1.0
4,0.866025,0.500000,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,1.005046,...,46.674281,52.189555,47.257474,50.685501,50.334074,50.297334,0.0,0.0,0.0,1.0


In [21]:
df_result.columns

Index(['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'mon_sin', 'mon_cos',
       'weekday_sin', 'weekday_cos', 'year', 'RATIO_close_and_MA3',
       'RATIO_close_and_MA6', 'RATIO_close_and_MA12', 'RATIO_close_and_MA24',
       'RATIO_close_and_MA48', 'RATIO_close_and_MA96', 'RATIO_close_and_MA192',
       'RATIO_close_and_MA384', 'RATIO_quote_asset_volume_and_MA3',
       'RATIO_quote_asset_volume_and_MA6', 'RATIO_quote_asset_volume_and_MA12',
       'RATIO_quote_asset_volume_and_MA24',
       'RATIO_quote_asset_volume_and_MA48',
       'RATIO_quote_asset_volume_and_MA96',
       'RATIO_quote_asset_volume_and_MA192',
       'RATIO_quote_asset_volume_and_MA384', 'RATIO_num_trades_and_MA3',
       'RATIO_num_trades_and_MA6', 'RATIO_num_trades_and_MA12',
       'RATIO_num_trades_and_MA24', 'RATIO_num_trades_and_MA48',
       'RATIO_num_trades_and_MA96', 'RATIO_num_trades_and_MA192',
       'RATIO_num_trades_and_MA384', 'RATIO_close_and_STD3',
       'RATIO_close_and_STD6', 'RATIO_close_a

In [22]:
max_val_PR = 0.65

for time_step in TIME_FRAME:
    df_tensor = dataframe_to_tensor(df_result, time_step)
    x_train, y_train, x_test, y_test = get_train_test_sets(df_tensor, TRAIN_FRACTION)
    
    df_tensor.shape

    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)
    model, history = model_design.model_design(EPOCHS,  x_train, y_train, x_test, y_test, class_weights_dict)

    val_PR = round(max(history.history['val_PR']),3)
    
    x = datetime.datetime.now()
    
    model_name = "models/" + x.strftime("%d-%m-%Y %H-%M-%S") +" val_PR " + str(val_PR) + " time frame " + str(time_step) + ".hd5"

    if val_PR > max_val_PR:
        max_val_PR = val_PR
        model.save(model_name,  save_format = 'h5')
    
    print(model_name)
    print(val_PR)
    print(150 * "-")


(26207, 2, 65)
(26207, 4)
(8736, 2, 65)
(8736, 4)


C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


models/22-01-2024 18-42-50 val_PR 0.653 time frame 2.hd5
0.653
------------------------------------------------------------------------------------------------------------------------------------------------------
(26206, 3, 65)
(26206, 4)
(8736, 3, 65)
(8736, 4)
models/22-01-2024 18-46-48 val_PR 0.643 time frame 3.hd5
0.643
------------------------------------------------------------------------------------------------------------------------------------------------------
(26205, 4, 65)
(26205, 4)
(8736, 4, 65)
(8736, 4)
models/22-01-2024 18-51-06 val_PR 0.623 time frame 4.hd5
0.623
------------------------------------------------------------------------------------------------------------------------------------------------------


In [23]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 4, 65)]           0         
                                                                 
 normalization_2 (Normaliza  (None, 4, 65)             131       
 tion)                                                           
                                                                 
 bidirectional_4 (Bidirecti  (None, 4, 130)            68120     
 onal)                                                           
                                                                 
 bidirectional_5 (Bidirecti  (None, 130)               101920    
 onal)                                                           
                                                                 
 flatten_2 (Flatten)         (None, 130)               0         
                                                           